In [136]:
import numpy as np
import json
import pandas as pd
import random
import pickle


In [138]:
types = ["car", "bus", "van", "pedestrian","dolly"];
x = np.random.randint(20,100, (10,4))
values_frames = []
print(x)

[[28 75 92 41]
 [42 36 70 97]
 [78 74 21 29]
 [44 44 44 69]
 [78 35 52 68]
 [71 63 63 89]
 [34 84 56 70]
 [44 25 85 69]
 [61 33 91 79]
 [72 55 95 38]]


In [139]:
a = random.randint(1,2)
b = random.randrange(0,4,1)
print(a)
print(b)

1
0


In [140]:
total_filename = 500
values = []

for i in range(total_filename):
    file_name = "name" + str(i+1)
    
    random_list = np.random.randint(5)
    types_len = len(types)
    typenum_list = np.random.randint(types_len, size=(random_list))
#     print(typenum_list)
    type_list = [types[value] for value in typenum_list]
    
    points = np.random.randint(0, 600, size=(random_list))
    occluded = np.random.randint(2,size=random_list)
    value = { "filename": file_name, "occluded": list(occluded), "points": list(points), "types": list(type_list)}
    values.append(value)
# print(values)
filename="data_result.pickle"
# values = {"values": values}
# print("Values", values)
with open(filename, 'wb') as f:
    pickle.dump(values, f)
    

In [141]:
filename = "data_result.pickle"

with open (filename,'rb') as f:
    data = pickle.load(f)
#create a series panda values with headers and columns
headers = list(data[0].keys())
print(headers)

# print(data)



['filename', 'occluded', 'points', 'types']


In [142]:
points_cl = []
types_cl = []
filenames_cl = []
occluded_cl = []

points_cl = np.concatenate([np.array(value["points"]) for value in data], axis=0)
types_cl =  np.concatenate([np.array(value["types"]) for value in data], axis=0)   
occluded_cl =  np.concatenate([np.array(value["occluded"]) for value in data], axis=0)   
filenames_cl = np.concatenate([np.full(len(value["types"]), value["filename"]) for value in data], axis=0)   

In [143]:
print("points shape",points_cl.shape)
print("type shape", types_cl.shape)
print("filename shape", filenames_cl.shape)
print("ocluded shape", occluded_cl.shape)

points shape (964,)
type shape (964,)
filename shape (964,)
ocluded shape (964,)


In [144]:
d = {"types": types_cl, "filename": filenames_cl, "occluded": occluded_cl, "points": points_cl}

In [75]:
print(d)

{'types': array(['bus', 'dolly', 'car', ..., 'bus', 'dolly', 'car'], dtype='<U32'), 'filename': array(['name1', 'name1', 'name2', ..., 'name499', 'name499', 'name500'],
      dtype='<U7'), 'occluded': array([0., 1., 1., ..., 0., 1., 0.]), 'points': array([165., 444., 129., ..., 561., 474., 147.])}


In [145]:
df = pd.DataFrame(data=d)

In [146]:
df

,types,filename,occluded,points
0,car,name1,0.0,494.0
1,van,name2,0.0,322.0
2,car,name2,1.0,113.0
3,bus,name3,0.0,72.0
4,van,name6,0.0,388.0
...,...,...,...,...
959,bus,name497,1.0,172.0
960,car,name497,0.0,62.0
961,dolly,name497,1.0,50.0
962,van,name498,0.0,293.0


In [128]:
#detect outlier
def detect_outlier_by_IQR(DataFrame, AttList, Rate=3.0):
    OutlierIdx = []
    for att in AttList:
        AttData = DataFrame.loc[:, att]
        lowerq = AttData.quantile(0.25)
        upperq = AttData.quantile(0.75)
        IQR = upperq - lowerq
        threshold_upper = (IQR*Rate) + upperq
        threshold_lower = lowerq - (IQR*Rate)
        #get list of idx that are outliers
        print("Min {}, Max {}, and IQR {}".format(AttData.min(), AttData.max(), IQR))
        print("Upper fence {} and lower fence {}".format(threshold_upper, threshold_lower))

        thres_data = AttData[AttData.apply(lambda x: x >threshold_upper or x < threshold_lower)]
        print(thres_data)
        
        AttOutlierIdx = set(thres_data.index.array)
        OutlierIdx = set(OutlierIdx) | AttOutlierIdx
        
        print(att + " " + str(len(AttOutlierIdx)) + " Outlier Idx: " + str(AttOutlierIdx))
    
    OutlierIdx = list(OutlierIdx)
    OutlierIdx = sorted(OutlierIdx)
    return OutlierIdx
        

In [203]:
from sklearn.neighbors import LocalOutlierFactor
def detect_outlier_by_LOF(data_frame, att_list, lof_threshold=3.0, neighbors=10):
    lof = LocalOutlierFactor(n_neighbors=neighbors)
    att_data = data_frame.loc[:, att_list].values
    y_pred = lof.fit_predict(att_data)
    #more than 0 is outlier
    attdata_scores = -1 * lof.negative_outlier_factor_
    lof_factor_data = pd.DataFrame(attdata_scores, columns=['LOF'])
#     print("LOF factor data", lof_factor_data)
    lof_factor_data = lof_factor_data.sort_values('LOF', ascending=False)
    lof_factor_data = lof_factor_data.reset_index(drop=False)
    suspect_outlier_data = lof_factor_data[lof_factor_data['LOF'].apply(lambda x: x > lof_threshold)]
    outlier_idx_list = suspect_outlier_data.index.array
    print("outlier len: " + str(len(outlier_idx_list)) + "Outlier idx: " + str(outlier_idx_list))
    return list(outlier_idx_list), lof_factor_data
    
    
    

In [192]:
def separate_data_by_att(DataFrame, att):
    AttData = DataFrame.loc[:,att]
    print(AttData)
    types = pd.Series(np.array(AttData)).unique()
    item_list = []
    for t in types:
        df_t = DataFrame[DataFrame[att] == t]
#         df_t = df_t.reset_index(drop=False)
        item_list.append(df_t)
        print("Data", df_t)
    print("Types: ",types)
    return item_list
    

In [193]:
#separate by class
outlier_index = set()
headers = df.columns.values.tolist()
print(headers)
item_list = separate_data_by_att(df,'types')


['types', 'filename', 'occluded', 'points']
0        car
1        van
2        car
3        bus
4        van
       ...  
959      bus
960      car
961    dolly
962      van
963      car
Name: types, Length: 964, dtype: object
Data     types filename  occluded  points
0     car    name1       0.0   494.0
2     car    name2       1.0   113.0
6     car    name6       0.0    67.0
9     car    name9       0.0   553.0
10    car   name10       0.0   130.0
..    ...      ...       ...     ...
956   car  name495       0.0    35.0
957   car  name495       0.0   436.0
958   car  name497       0.0   454.0
960   car  name497       0.0    62.0
963   car  name498       1.0   121.0

[196 rows x 4 columns]
Data     types filename  occluded  points
1     van    name2       0.0   322.0
4     van    name6       0.0   388.0
12    van   name10       1.0     6.0
24    van   name16       0.0   462.0
25    van   name16       0.0   579.0
..    ...      ...       ...     ...
925   van  name478       1.0   113.0

In [204]:
outlierAtt = ['points']
outlier_idx_list = []
for item in item_list:
    print("__________________________")
    outlier_idx_list += detect_outlier_by_IQR(item, AttList=outlierAtt, Rate=0.5)
    print("__________________________")
    outlier_idx_lof, lof_factor_data = detect_outlier_by_LOF(item, outlierAtt, 3.0, 10)
    outlier_idx_list += outlier_idx_lof
    print("size of outlier ", outlier_idx_lof)
    print(lof_factor_data.head())
        


#     print("Size of IQR: " + str(len(outlier_idx)))
#remove outlier by LOF as well


__________________________
Min 2.0, Max 595.0, and IQR 294.75
Upper fence 579.125 and lower fence -10.375
770    595.0
Name: points, dtype: float64
points 1 Outlier Idx: {770}
__________________________
outlier len: 0Outlier idx: <PandasArray>
[]
Length: 0, dtype: int64
size of outlier  []
   index       LOF
0    156  1.651607
1     89  1.503964
2     86  1.469283
3    109  1.425988
4    183  1.354740
__________________________
Min 3.0, Max 599.0, and IQR 341.0
Upper fence 643.5 and lower fence -38.5
Series([], Name: points, dtype: float64)
points 0 Outlier Idx: set()
__________________________
outlier len: 0Outlier idx: <PandasArray>
[]
Length: 0, dtype: int64
size of outlier  []
   index       LOF
0     78  1.655533
1    146  1.525723
2     68  1.476271
3    145  1.476271
4    143  1.470047
__________________________
Min 5.0, Max 599.0, and IQR 323.0
Upper fence 582.5 and lower fence -63.5
38     595.0
127    599.0
434    598.0
488    592.0
489    586.0
Name: points, dtype: float64
p

In [205]:
#Remove outliers idx 
print(outlier_idx_list)
print(df.loc[[770]])
print(df.loc[[645]])
print(df.loc[[434]])

[770, 38, 127, 434, 488, 489, 63, 198, 237, 645]
    types filename  occluded  points
770   car  name398       0.0   595.0
     types filename  occluded  points
645  dolly  name330       1.0   599.0
    types filename  occluded  points
434   bus  name228       0.0   598.0


In [206]:
def remove_rows_from_dataframe(dataframe, rowIdxList = []):
    data_frame = dataframe.drop(rowIdxList)
    print(data_frame.shape)
    data_frame = data_frame.reset_index(drop=True)
    return data_frame

In [211]:
#remove data frome data_frame should we cluster points inside box?
data_frame = remove_rows_from_dataframe(df,outlier_idx_list)

(954, 4)


In [212]:
print(data_frame.shape)

(954, 4)


In [226]:
#now convert back to original form
def convert_to_group_filename(data_frame):
    AttData = data_frame.loc[:,'filename']
    print(AttData)
    names = pd.Series(np.array(AttData)).unique()
    object_list = []
    for name in names:
        df_t = data_frame[data_frame['filename'] == name]
        print("DF_T", df_t)
        types = df_t['types'].tolist()
        occluded = df_t['occluded'].tolist()
        points = df_t['points'].tolist()
        value = {'filename':name, "types": types, "occluded": occluded, "points": points}
        object_list.append(value)
    return object_list

In [227]:
object_list = convert_to_group_filename(data_frame)
print("Object List", object_list)

0        name1
1        name2
2        name2
3        name3
4        name6
        ...   
949    name497
950    name497
951    name497
952    name498
953    name498
Name: filename, Length: 954, dtype: object
DF_T   types filename  occluded  points
0   car    name1       0.0   494.0
DF_T   types filename  occluded  points
1   van    name2       0.0   322.0
2   car    name2       1.0   113.0
DF_T   types filename  occluded  points
3   bus    name3       0.0    72.0
DF_T   types filename  occluded  points
4   van    name6       0.0   388.0
5   bus    name6       1.0   325.0
6   car    name6       0.0    67.0
DF_T         types filename  occluded  points
7         bus    name9       0.0    88.0
8  pedestrian    name9       1.0   375.0
9         car    name9       0.0   553.0
DF_T     types filename  occluded  points
10    car   name10       0.0   130.0
11  dolly   name10       1.0    93.0
12    van   name10       1.0     6.0
DF_T          types filename  occluded  points
13  pedestrian   n

190         car  name108       1.0   283.0
DF_T           types filename  occluded  points
191  pedestrian  name109       1.0   576.0
192  pedestrian  name109       1.0    12.0
DF_T     types filename  occluded  points
193   van  name110       1.0    24.0
194   car  name110       1.0     2.0
DF_T      types filename  occluded  points
195  dolly  name112       0.0   187.0
196    van  name112       0.0    89.0
197    van  name112       1.0   467.0
DF_T           types filename  occluded  points
198       dolly  name113       0.0   227.0
199         van  name113       0.0   408.0
200  pedestrian  name113       0.0   312.0
201         bus  name113       1.0   277.0
DF_T           types filename  occluded  points
202         bus  name115       0.0   131.0
203  pedestrian  name115       0.0    28.0
204  pedestrian  name115       1.0   332.0
DF_T     types filename  occluded  points
205   car  name116       0.0    70.0
DF_T     types filename  occluded  points
206   car  name117       1.0   3

407         car  name216       0.0   337.0
DF_T           types filename  occluded  points
408         van  name217       1.0   149.0
409  pedestrian  name217       1.0   342.0
DF_T      types filename  occluded  points
410    car  name218       0.0   152.0
411  dolly  name218       1.0   557.0
412  dolly  name218       1.0   292.0
DF_T           types filename  occluded  points
413  pedestrian  name219       0.0   512.0
414       dolly  name219       0.0   393.0
415  pedestrian  name219       1.0     6.0
DF_T     types filename  occluded  points
416   bus  name221       1.0   514.0
DF_T     types filename  occluded  points
417   bus  name223       0.0   327.0
418   van  name223       0.0    65.0
419   van  name223       1.0   549.0
420   car  name223       0.0   238.0
DF_T           types filename  occluded  points
421  pedestrian  name224       1.0    18.0
422         car  name224       1.0   561.0
423       dolly  name224       1.0   497.0
424       dolly  name224       1.0   284.0


623  pedestrian  name322       1.0    69.0
DF_T           types filename  occluded  points
624         van  name324       0.0   397.0
625  pedestrian  name324       1.0   166.0
626  pedestrian  name324       1.0   100.0
DF_T           types filename  occluded  points
627       dolly  name325       0.0   186.0
628         car  name325       1.0   270.0
629         bus  name325       1.0    53.0
630  pedestrian  name325       1.0   108.0
DF_T           types filename  occluded  points
631         car  name326       1.0   369.0
632         car  name326       1.0   173.0
633  pedestrian  name326       0.0   467.0
DF_T     types filename  occluded  points
634   bus  name328       0.0   115.0
635   van  name328       0.0   400.0
DF_T     types filename  occluded  points
636   van  name330       0.0    38.0
637   van  name330       0.0   531.0
DF_T           types filename  occluded  points
638  pedestrian  name331       0.0   453.0
639       dolly  name331       1.0   507.0
640         car  

821   car  name433       1.0   133.0
DF_T     types filename  occluded  points
822   van  name434       0.0   291.0
DF_T     types filename  occluded  points
823   bus  name435       0.0   211.0
824   bus  name435       1.0    76.0
DF_T     types filename  occluded  points
825   bus  name436       1.0    24.0
826   van  name436       1.0   277.0
DF_T     types filename  occluded  points
827   car  name437       1.0   434.0
DF_T      types filename  occluded  points
828    van  name439       1.0   500.0
829  dolly  name439       0.0    88.0
DF_T     types filename  occluded  points
830   car  name440       0.0   127.0
831   van  name440       1.0   206.0
832   car  name440       0.0   450.0
DF_T           types filename  occluded  points
833  pedestrian  name441       0.0   113.0
834         van  name441       1.0   393.0
835         bus  name441       0.0   334.0
DF_T      types filename  occluded  points
836  dolly  name442       0.0   283.0
837    car  name442       0.0   447.0
DF_T 

In [228]:
#pickle file
with open('new_data.pkl', 'wb') as f:
    pickle.dump(object_list, f)